# Lock exchange flow example
This is a test case with particles released in a simple looping flow field 
The background flow velocity is given by a lock exchange flow.

In [22]:
# Load required libraries
using Pkg
Pkg.instantiate()
Pkg.activate("..")
using Particles
using Plots
using Random

randpool = MersenneTwister(0) #Generate same random numbers every time

nothing

  Activating project at `i:\Master_Thesis\particles.jl`


In [23]:
# load useful default settings 
d=default_userdata()
n=0 #number of particles

d["nparticles"]=n
d["plot_maps_size"] = (2000,300)
d["time_direction"] = :forwards # :forwards or :backwards

dflow_map = load_nc_info(@__DIR__, r"locxx_map.nc")

# does the following function calculate the whole grid with same resolution
# in each direction?
interp = load_dflow_grid(dflow_map, 50, false);

# keep some output in memory
# no clue why a 1 should be used here; disk array indexing
d["keep_particles"] = true #keep results in memory (bad idea for a large run)
d["keep_particle_times"] = dflow_map[1]["time"][:]
nothing


compute index:
- i:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc


┌ Info: i:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc
└ @ Particles i:\Master_Thesis\particles.jl\src\dflow.jl:27


In [30]:
# set the reference(starting) time with default setting
# no need for the 3rd dimension 
u = initialize_interpolation(dflow_map, interp, "mesh2d_ucx", d["reftime"], 0.0, d["time_direction"]);
v = initialize_interpolation(dflow_map, interp, "mesh2d_ucy", d["reftime"], 0.0, d["time_direction"]);
display(typeof(u))


Particles.var"#f#20"{Interpolator, Float64, Symbol, Particles.var"#weights#19", Particles.var"#update_cache_backwards#17"{Vector{Any}}, Particles.var"#update_cache_forwards#14"{Vector{Int64}, Vector{Any}, Bool, Int64}, Vector{Any}}

initialize caching for i:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucx...
initialize caching for i:\Master_Thesis\particles.jl\case_lock_exchange\locxx_map.nc mesh2d_ucy...


In [29]:
# prepare the grid for ploting
# use square grid
x_resolution=601 #[0,300]
y_resolution=3   #[0,1]
x = collect(range(minimum(dflow_map[1]["mesh2d_node_x"][:]), maximum(dflow_map[1]["mesh2d_node_x"][:]), length=x_resolution))
y = collect(range(minimum(dflow_map[1]["mesh2d_node_y"][:]), maximum(dflow_map[1]["mesh2d_node_y"][:]), length=y_resolution))

xs=kron(x,ones(length(y)))
ys=kron(ones(length(x)),y)
Plots.default(:size, d["plot_maps_size"])
u_plot = zeros(length(x)*length(y))
v_plot = zeros(length(x)*length(y))
for i = 1:length(x)
    for j = 1:length(y)
        u_plot[(i-1)*length(y)+j] = u(x[i], y[j], 0.0, 0.0)
        v_plot[(i-1)*length(y)+j] = v(x[i], y[j], 0.0, 0.0)
    end
end
quiver(xs,ys,quiver=(u_plot,v_plot))
savefig("bkg.png")

ErrorException: Trying to access before first map t=0.0 < 3.16224e7

In [6]:
# Here is the equation that we want to solve for the particles

"""
   !f(ds,s,t,i,d)

Dynamic model, computes time derivative ds of s at current time t
for particle i and possibly using data/functions from d of type userdata.
"""
function f!(∂s, s, t, i, d)
   x, y, z, age = s
   # dx/dt=u
    ∂s.x = u(x, y, z, t)
   # dy/dt=v
    ∂s.y = v(x, y, z, t)
   # dz/dt=0
    ∂s.z = 0.0
   # age=(t-t0)
    ∂s.t = 1.0
end
#d["f"] = f!

f!

In [7]:
###### Velocity function for the particles ######
function f1!(ds, s, t, i, d)
   x, y, z, age = s
   z = 0.0
                                                             # Converts radians to degrees
   up = 0
   vp = 0
   dt = d["dt"]
   uw = u(x, y, z, t)
   vw = v(x, y, z, t)

   # Various models:
   # 0: Use drifer data
   # track_of_drifter!(ds,s,t,d["reftime"],d["dt"],drifter)
   # 1: Only flow velocities
   up = uw
   vp = vw
   # 2: Flow plus a factor times wind
   # up = uw+0.016*ua*0.9
   # vp = vw+0.016*va*0.9
   # 3: Add stokes drift to flow velocity
   # usJ, vsJ = uv_sJ(wh(x,y,z,t),wp(x,y,z,t),wd(x,y,z,t))
   # up = uw+usJ
   # vp = vw+vsJ
   # 4: Combine flow, stokes drift and wind in an equilibrium for the particle velocity
   # usJ, vsJ = uv_sJ(wh(x,y,z,t),wp(x,y,z,t),wd(x,y,z,t))
   # (up,vp) = water_stokes_wind(ua,va, uw,vw,usJ,vsJ)

   # Calculate and add turbulent diffusivity, using Pr=1
   # Estimate the Eddy viscosity and its derivates, using a Smagorinsky model
   # This is only because the horizontal diffusion is not in the flow output files
   
   # original computation for computing the particle velocity
   #(K, Kdx, Kdy) = estimate_viscosity_smag(interp, x, y, t, u, v)
   #if !(uw == vw == ua == va == 0.0)
   #   # https://doi.org/10.1016/j.ocemod.2017.11.008 eq. 27
   #   up += Kdy + randn() * sqrt(2 * K * dt) / dt
   #   vp += Kdx + randn() * sqrt(2 * K * dt) / dt
   #end
   epsx = 1
   epsz = 1e-5
   up += randn()*sqrt(2*epsx*dt)/dt
   vp += randn()*sqrt(2*epsz*dt)/dt

   ds.x = up
   ds.y = vp
   ds.z = 0.0
   ds.t = 1.0


   if d["time_direction"] == :backwards
      up *= -1
      vp *= -1
   end
end
d["f"]=f1!

f1! (generic function with 1 method)

In [8]:
# use streamfunction as background for plotting

function plot_background(d)
    x = 0:5:100
    y = 0:5:50
    xs=kron(x,ones(length(y)))
    ys=kron(ones(length(x)),y)
    Plots.default(:size, d["plot_maps_size"])
    scale = 5
    u_plot = zeros(length(x)*length(y))
    v_plot = zeros(length(x)*length(y))
    for i = 1:length(x)
        for j = 1:length(y)
            u_plot[(i-1)*length(y)+j] = u(x[i], y[j], 0.0, 0.0)
            v_plot[(i-1)*length(y)+j] = v(x[i], y[j], 0.0, 0.0)
        end
    end
    f = quiver(xs,ys,quiver=(scale*u_plot,scale*v_plot))
    return(f)
 end
 d["plot_maps_background"] = plot_background
 

plot_background (generic function with 1 method)

In [ ]:
#run the actual particle model

@time run_simulation(d)

In [10]:
d["plot_maps_func"]

plot_maps_xy (generic function with 1 method)

In [20]:
# screenshot of particles
t_plot=d["keep_particle_times"]
p=d["all_particles"]
for i in 1:length(t_all)
    fig = d["plot_maps_background"](d)
    fname = "vortex\\screenshot_time"*"$(t_plot[i])"*".png"
    d["plot_maps_func"](fig, d, p[i])
    savefig(fig, fname)
end


In [51]:
display(size(p))
display(typeof(p))
display(size(p[1]))
display(size(p[:][:,1]))
display(size(p[:][1,1]))
display(typeof(p[:][1,1]))

display(size(p[1][1,:]))
display(typeof(p[1][1,:]))

(101,)

Vector{Any} (alias for Array{Any, 1})

(4, 30)

(101,)

(4, 30)

Matrix{Float64} (alias for Array{Float64, 2})

(30,)

Vector{Float64} (alias for Array{Float64, 1})

In [29]:
# obtain the location data
display(d["variables"])
display(length(t_plot))
display("Particle number")
display(n)

4-element Vector{String}:
 "x"
 "y"
 "z"
 "age"

101

"Particle number"

30

In [59]:
p_x = zeros(length(t_plot), n)
p_y = zeros(length(t_plot), n)
for i in 1:length(t_plot)
    p_x[i, :] = p[i][1,:]
    p_y[i, :] = p[i][2,:]
end

# prepare the data for plotting
# note that the data points at the first and the end are used once
indices = reduce(vcat, [[j for i in 1:2] for j in 1:length(t_plot)])
indices = indices[begin+1:end-1]
fig = d["plot_maps_background"](d)
dname = "loccx.png"

for i in 1:n
    plot!(fig, p_x[indices, i], p_y[indices, i], legend=egend =:outerright, size = (1400, 800))
end

savefig(fig, fname)


"i:\\Master_Thesis\\particles.jl\\example_notebooks\\vortex\\trajectories.png"